# Chapter 3: Implementation Details

This chapter explains how distfeat works internally, from data structures to algorithms. Understanding these details helps users extend the library and researchers reproduce the methods.

## Architecture Overview

distfeat follows a modular design with clear separation of concerns:

In [ ]:
# Import the main modules to show the structure
import distfeat
from distfeat import features, distances, normalization, io, config

print("distfeat module structure:")
print(f"  features:      {len([x for x in dir(features) if not x.startswith('_')])} public functions")
print(f"  distances:     {len([x for x in dir(distances) if not x.startswith('_')])} public functions")
print(f"  normalization: {len([x for x in dir(normalization) if not x.startswith('_')])} public functions")
print(f"  io:            {len([x for x in dir(io) if not x.startswith('_')])} public functions")
print(f"  config:        {len([x for x in dir(config) if not x.startswith('_')])} public functions")

print("\nMain API functions:")
api_functions = [name for name in dir(distfeat) if not name.startswith('_')]
for func in api_functions[:10]:  # Show first 10
    print(f"  - {func}")
if len(api_functions) > 10:
    print(f"  ... and {len(api_functions) - 10} more")

## Feature System Implementation

### Data Loading and Caching

The feature system uses a global cache for efficiency:

In [ ]:
from distfeat.features import _initialize_features, get_feature_names, get_feature_system
import time

# Time the initialization
start_time = time.time()
_initialize_features()
init_time = time.time() - start_time

print(f"Feature system initialization: {init_time:.3f} seconds")

# Show the cache structure
feature_system = get_feature_system()
feature_names = get_feature_names()

print(f"\nCached data:")
print(f"  Phonemes: {len(feature_system)}")
print(f"  Features: {len(feature_names)}")
print(f"  Memory usage: ~{len(feature_system) * len(feature_names) * 4 / 1024:.1f} KB")

# Show feature names
print(f"\nFeature types:")
feature_categories = {
    'Major class': ['consonantal', 'sonorant', 'syllabic'],
    'Place': ['labial', 'coronal', 'dorsal', 'pharyngeal'],
    'Manner': ['continuant', 'nasal', 'lateral', 'strident'],
    'Voicing': ['voice', 'spread', 'constricted']
}

for category, examples in feature_categories.items():
    available = [f for f in examples if f in feature_names]
    print(f"  {category}: {available}")

### Feature Vector Representation

Phonemes are represented as binary feature vectors:

In [ ]:
from distfeat import phoneme_to_features
import numpy as np

# Examine feature vectors for different sound types
sound_examples = {
    'Voiceless stop': 'p',
    'Voiced stop': 'b', 
    'Fricative': 'f',
    'Nasal': 'm',
    'Vowel': 'a'
}

print("Feature vector examples:")
print("(Showing first 10 features only)\n")

# Get feature names for display
feature_names = get_feature_names()
display_features = feature_names[:10]

# Header
print(f"{'Sound':15}", end="")
for feat in display_features:
    print(f"{feat[:8]:>9}", end="")
print()
print("-" * (15 + 9 * len(display_features)))

# Feature vectors
for sound_type, phoneme in sound_examples.items():
    features = phoneme_to_features(phoneme)
    if features:
        print(f"{sound_type} [{phoneme}]:    ", end="")
        for feat in display_features:
            value = features.get(feat, 0)
            print(f"{value:9}", end="")
        print()

print(f"\nTotal features per phoneme: {len(feature_names)}")
print(f"Storage: {len(feature_names)} bits = {len(feature_names) // 8} bytes per phoneme")

### Optimizations

Several optimizations improve performance:

In [ ]:
from distfeat import calculate_distance
from functools import lru_cache
import time

# Demonstrate caching performance
phoneme_pairs = [('p', 'b'), ('t', 'd'), ('k', 'g')] * 100

# Time with cache
start_time = time.time()
for p1, p2 in phoneme_pairs:
    calculate_distance(p1, p2)
cached_time = time.time() - start_time

print(f"Cached distance calculations:")
print(f"  300 distance calls: {cached_time:.3f} seconds")
print(f"  ~{300/cached_time:.0f} calls per second")

# Show cache info (internal)
print(f"\nCaching benefits:")
print(f"  - Feature vectors cached globally")
print(f"  - Distance calculations use LRU cache")
print(f"  - NumPy vectorization for matrix operations")
print(f"  - Lazy initialization of feature system")

## Distance Calculation Implementation

### Core Distance Functions

Each distance metric has a specific implementation:

In [ ]:
from distfeat.distances import (
    _hamming_distance, _jaccard_distance, _euclidean_distance,
    _cosine_distance, _manhattan_distance
)
import numpy as np

# Create example feature vectors
vec1 = np.array([1, 0, 1, 0, 1])  # Pattern: 10101
vec2 = np.array([1, 1, 0, 0, 1])  # Pattern: 11001

print("Distance implementations:")
print(f"Vector 1: {vec1}")
print(f"Vector 2: {vec2}")
print()

# Calculate each distance
hamming = _hamming_distance(vec1, vec2, normalize=True)
jaccard = _jaccard_distance(vec1, vec2)
euclidean = _euclidean_distance(vec1, vec2, normalize=True)
cosine = _cosine_distance(vec1, vec2)
manhattan = _manhattan_distance(vec1, vec2, normalize=True)

print(f"Hamming distance:   {hamming:.3f} (differs in {np.sum(vec1 != vec2)}/5 positions)")
print(f"Jaccard distance:   {jaccard:.3f} (1 - intersection/union)")
print(f"Euclidean distance: {euclidean:.3f} (L2 norm, normalized)")
print(f"Cosine distance:    {cosine:.3f} (1 - cosine similarity)")
print(f"Manhattan distance: {manhattan:.3f} (L1 norm, normalized)")

print("\nImplementation details:")
print("  - All distances normalized to [0,1] range")
print("  - Hamming: simple XOR operation")
print("  - Jaccard: set intersection/union on active features")
print("  - Euclidean/Manhattan: standard Lp norms")
print("  - Cosine: handles zero vectors gracefully")

### K-means Clustering Distance

The K-means method is more complex, involving clustering and centroid distances:

In [ ]:
from distfeat import build_distance_matrix
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np

# Demonstrate K-means clustering process
phonemes = ['p', 'b', 't', 'd', 'k', 'g', 'm', 'n', 'ŋ', 'f', 's', 'x']

print("K-means distance calculation process:")
print("=====================================")

# Step 1: Get feature vectors
feature_vectors = []
for p in phonemes:
    features = phoneme_to_features(p)
    if features:
        vec = [features.get(f, 0) for f in feature_names]
        feature_vectors.append(vec)

X = np.array(feature_vectors)
print(f"Step 1: Feature matrix shape: {X.shape}")

# Step 2: K-means clustering  
n_clusters = 4
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X)
centroids = kmeans.cluster_centers_

print(f"Step 2: K-means with {n_clusters} clusters")
print(f"        Inertia: {kmeans.inertia_:.2f}")

# Show cluster assignments
print("\nStep 3: Cluster assignments:")
for i, (phoneme, cluster_id) in enumerate(zip(phonemes, clusters)):
    print(f"  [{phoneme}] → Cluster {cluster_id}")

# Step 4: Calculate centroid distances
from sklearn.metrics import pairwise_distances
centroid_distances = pairwise_distances(centroids)
max_dist = np.max(centroid_distances)
normalized_distances = centroid_distances / max_dist if max_dist > 0 else centroid_distances

print(f"\nStep 4: Centroid distance matrix:")
print("   ", end="")
for i in range(n_clusters):
    print(f"C{i:2}", end="")
print()
for i in range(n_clusters):
    print(f"C{i} ", end="")
    for j in range(n_clusters):
        print(f"{normalized_distances[i,j]:5.2f}", end="")
    print()

# Final result
print(f"\nStep 5: Final phoneme distances based on cluster membership")
print(f"        Same cluster → distance 0")
print(f"        Different clusters → distance from centroid matrix")

### Distance Matrix Construction

Building full distance matrices efficiently:

In [ ]:
import time

# Time matrix construction
test_phonemes = ['p', 'b', 't', 'd', 'k', 'g', 'f', 'v', 's', 'z']
n = len(test_phonemes)

print(f"Distance matrix construction for {n} phonemes:")
print(f"Matrix size: {n}×{n} = {n*n} cells")
print(f"Unique distances: {n*(n-1)//2} (due to symmetry)")

# Time different methods
methods = ['hamming', 'jaccard', 'euclidean', 'kmeans']
for method in methods:
    start_time = time.time()
    matrix, labels = build_distance_matrix(test_phonemes, method=method)
    elapsed = time.time() - start_time
    
    unique_values = len(np.unique(matrix))
    print(f"  {method:10}: {elapsed:.3f}s, {unique_values:2d} unique distances")

print("\nOptimizations:")
print("  - Symmetric matrices: compute upper triangle only")
print("  - Vectorized operations where possible")
print("  - Caching of feature lookups")
print("  - K-means: single clustering, reuse centroids")

## Normalization Implementation

### Unicode Normalization Pipeline

IPA normalization involves several steps:

In [ ]:
from distfeat.normalization import normalize_ipa, normalize_glyph
import unicodedata

# Demonstrate normalization steps
test_cases = [
    "pʰæ̃n",        # Aspirated, nasalized
    "t̪ʰin",        # Dental, aspirated
    "kʷā",         # Labialized, long
    "ʃi:",          # ASCII length mark
    "tɕʰi",        # Complex consonant + aspiration
]

print("IPA Normalization Pipeline:")
print("==========================\n")

for original in test_cases:
    print(f"Original:  '{original}'")
    print(f"  Length: {len(original)} chars")
    
    # Show Unicode decomposition
    nfd = unicodedata.normalize('NFD', original)
    print(f"  NFD:    '{nfd}' ({len(nfd)} chars)")
    
    # Full normalization
    normalized = normalize_ipa(original)
    print(f"  Final:  '{normalized}' ({len(normalized)} chars)")
    
    # Character breakdown
    chars = []
    for char in normalized:
        name = unicodedata.name(char, f'U+{ord(char):04X}')
        chars.append(f'{char}({name[:20]}...)')
    print(f"  Chars:  {' + '.join(chars)}")
    print()

print("Normalization steps:")
print("  1. NFD decomposition (separate base + diacritics)")
print("  2. Diacritic reordering (consistent order)")
print("  3. ASCII substitutions (: → ː)")
print("  4. Case normalization (optional)")
print("  5. Tone handling (preserve or remove)")

## I/O and Serialization

### Matrix Export Formats

distfeat supports multiple export formats:

In [ ]:
from distfeat import save_distance_matrix, load_distance_matrix
import tempfile
import os
import json

# Create a small test matrix
test_phonemes = ['p', 'b', 't']
matrix, labels = build_distance_matrix(test_phonemes)

print("Export format examples:")
print("======================\n")

with tempfile.TemporaryDirectory() as tmpdir:
    # TSV format
    tsv_path = os.path.join(tmpdir, 'test.tsv')
    save_distance_matrix(matrix, labels, tsv_path, format='tsv')
    
    with open(tsv_path, 'r') as f:
        tsv_content = f.read()
    print("TSV format (UNIPA compatible):")
    print(tsv_content)
    
    # JSON format
    json_path = os.path.join(tmpdir, 'test.json')
    save_distance_matrix(matrix, labels, json_path, format='json')
    
    with open(json_path, 'r') as f:
        json_data = json.load(f)
    print("JSON format (with metadata):")
    print(f"  phonemes: {json_data['phonemes']}")
    print(f"  matrix: {len(json_data['matrix'])}x{len(json_data['matrix'][0])}")
    print(f"  metadata: {json_data['metadata']}")
    print()
    
    # Test round-trip
    loaded_matrix, loaded_labels = load_distance_matrix(tsv_path)
    
    print("Round-trip test:")
    print(f"  Original shape: {matrix.shape}")
    print(f"  Loaded shape:   {loaded_matrix.shape}")
    print(f"  Max difference: {np.max(np.abs(matrix - loaded_matrix)):.6f}")
    print(f"  Labels match:   {labels == loaded_labels}")

print("\nSupported formats:")
print("  - TSV: tab-separated, compatible with UNIPA")
print("  - CSV: comma-separated, Excel-compatible")
print("  - JSON: with metadata, web-friendly")
print("  - NumPy: binary format for Python")

## Error Handling and Robustness

### Graceful Degradation

distfeat handles various error conditions:

In [ ]:
from distfeat import phoneme_to_features, calculate_distance
import logging

# Configure logging to see warnings
logging.basicConfig(level=logging.WARNING)

print("Error handling examples:")
print("========================\n")

# Test missing phoneme handling
print("1. Unknown phonemes:")
unknown_phonemes = ['zzz', '☃', 'invalid']
for phoneme in unknown_phonemes:
    # Default behavior: warn and return None
    result = phoneme_to_features(phoneme, on_error='warn')
    print(f"   '{phoneme}' → {result}")

print("\n2. Distance calculation with missing phonemes:")
valid_distance = calculate_distance('p', 'b')
invalid_distance = calculate_distance('p', 'zzz')
print(f"   p-b distance: {valid_distance}")
print(f"   p-zzz distance: {invalid_distance}")

print("\n3. Matrix operations with mixed valid/invalid:")
mixed_phonemes = ['p', 'b', 'invalid', 't']
try:
    matrix, labels = build_distance_matrix(mixed_phonemes)
    print(f"   Matrix shape: {matrix.shape}")
    print(f"   Non-zero entries: {np.count_nonzero(matrix)}")
    print(f"   Max distance: {np.max(matrix):.3f}")
except Exception as e:
    print(f"   Error: {e}")

print("\nError handling modes:")
print("  - 'warn': Log warning, return None (default)")
print("  - 'ignore': Silent failure, return None")
print("  - 'raise': Raise ValueError exception")
print("  - Invalid phonemes get max distance (1.0)")
print("  - Matrices handle partial data gracefully")

## Performance Considerations

### Scalability Analysis

Understanding performance characteristics:

In [ ]:
import time
import matplotlib.pyplot as plt

# Test scaling behavior
print("Performance scaling analysis:")
print("============================\n")

# Get available phonemes
feature_system = get_feature_system()
all_phonemes = list(feature_system.keys())

# Test different matrix sizes
sizes = [10, 20, 50, 100, 200]
times = []

print("Matrix size vs construction time:")
for size in sizes:
    if size <= len(all_phonemes):
        phonemes = all_phonemes[:size]
        
        start_time = time.time()
        matrix, _ = build_distance_matrix(phonemes, method='hamming')
        elapsed = time.time() - start_time
        times.append(elapsed)
        
        operations = size * (size - 1) // 2
        ops_per_sec = operations / elapsed if elapsed > 0 else 0
        
        print(f"  {size:3d} phonemes: {elapsed:.3f}s ({ops_per_sec:.0f} ops/sec)")
    else:
        times.append(None)

# Memory usage estimation
print(f"\nMemory usage estimates:")
feature_count = len(get_feature_names())
phoneme_count = len(all_phonemes)

feature_cache_mb = (phoneme_count * feature_count * 4) / (1024 * 1024)
matrix_1000_mb = (1000 * 1000 * 8) / (1024 * 1024)

print(f"  Feature cache: {feature_cache_mb:.1f} MB ({phoneme_count} phonemes × {feature_count} features)")
print(f"  1000×1000 matrix: {matrix_1000_mb:.1f} MB (float64)")

# Complexity analysis
print(f"\nComplexity analysis:")
print(f"  Single distance: O(F) where F = feature count")
print(f"  Distance matrix: O(N²·F) where N = phoneme count")
print(f"  K-means matrix: O(K·N·F·I) where K = clusters, I = iterations")
print(f"  Memory: O(N·F + N²) for cache + matrix")

print(f"\nOptimization strategies:")
print(f"  - Use symmetric matrices (store upper triangle only)")
print(f"  - Cache frequently accessed distances")
print(f"  - Batch process multiple phoneme pairs")
print(f"  - Consider sparse matrices for large, sparse datasets")

## Extension Points

### Adding Custom Distance Methods

Users can register custom distance functions:

In [ ]:
from distfeat import register_distance_method, calculate_distance
import numpy as np

# Define a custom weighted Hamming distance
def weighted_hamming(vec1, vec2):
    """Hamming distance with feature importance weighting."""
    # Example weights (in practice, learn from data)
    weights = np.linspace(0.5, 2.0, len(vec1))  # Later features weighted more
    differences = (vec1 != vec2).astype(float)
    weighted_diff = np.sum(weights * differences)
    return weighted_diff / np.sum(weights)  # Normalize

# Register the custom method
register_distance_method('weighted_hamming', weighted_hamming)

print("Custom distance method example:")
print("==============================\n")

# Test the custom method
pairs = [('p', 'b'), ('p', 't'), ('p', 'a')]
for p1, p2 in pairs:
    standard = calculate_distance(p1, p2, method='hamming')
    custom = calculate_distance(p1, p2, method='weighted_hamming')
    print(f"  {p1}-{p2}: standard={standard:.3f}, weighted={custom:.3f}")

print("\nCustom method requirements:")
print("  - Function signature: (vec1, vec2) -> float")
print("  - Input: numpy arrays of same length")
print("  - Output: non-negative distance value")
print("  - Register with register_distance_method()")
print("  - Works with all distfeat functions")

### Custom Feature Systems

Loading and using custom feature systems:

In [ ]:
# Create a minimal custom feature system
import tempfile
import csv

print("Custom feature system example:")
print("==============================\n")

# Create sample data
custom_data = [
    ['phoneme', 'vowel', 'high', 'back', 'round'],
    ['i', '1', '1', '0', '0'],
    ['u', '1', '1', '1', '1'], 
    ['a', '1', '0', '0', '0'],
    ['p', '0', '0', '0', '0'],
    ['k', '0', '0', '1', '0']
]

with tempfile.NamedTemporaryFile(mode='w', suffix='.csv', delete=False) as f:
    writer = csv.writer(f)
    writer.writerows(custom_data)
    custom_path = f.name

try:
    # Load custom system
    from distfeat import load_custom_features
    load_custom_features(
        custom_path,
        name='minimal',
        delimiter=',',
        phoneme_col='phoneme'
    )
    
    print("Custom system loaded successfully!")
    
    # Use custom system
    custom_features = phoneme_to_features('i', system='minimal')
    print(f"Features for 'i' in custom system: {custom_features}")
    
    # Calculate distance with custom system
    # Note: This would need system parameter support in calculate_distance
    print("\nCustom systems support:")
    print("  - CSV/TSV files with any delimiter")
    print("  - Flexible column naming")
    print("  - Binary or numeric features")
    print("  - Multiple systems simultaneously")
    
finally:
    os.unlink(custom_path)

## Summary

distfeat's implementation emphasizes:

1. **Performance**: Caching, vectorization, and optimized data structures
2. **Robustness**: Graceful error handling and input validation
3. **Extensibility**: Plugin architecture for custom methods and features
4. **Compatibility**: Multiple I/O formats and Unicode normalization
5. **Scalability**: Efficient algorithms for large phoneme inventories

The modular design allows users to leverage individual components while the unified API provides convenience for common tasks.

## Next Steps

- **Validation**: See how distfeat performs on real data in [Chapter 4](04_validation.ipynb)
- **Applications**: Explore use cases in the [Case Studies](../case_studies/indo_european.ipynb)
- **API Reference**: Detailed function documentation in [API docs](../api/features.ipynb)